Import the necessary libraries


In [ ]:
import numpy as np
import pandas as pd
import mne
import scipy.stats as stats
import pyentrp.entropy as ent
import pywt

DATA LOADING

Load the eegs of the first 5 participants into eeg_data dictionary

In [ ]:
participant_ids = [f"s{i:02d}" for i in range(1, 6)]  # Assuming 5 participants
eeg_data = {}

# Load EEG data for each participant
for participant_id in participant_ids:
    file_path = f"eegs/{participant_id}.bdf"
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    eeg_data[participant_id] = raw

# Print information about loaded data
for participant_id, raw in eeg_data.items():
    print(f"Participant ID: {participant_id}")
    # print(raw.info)

Keep only the eeg channels, reorder them to Geneva order and store them to eeg_data_reordered dict

In [ ]:
import mne

# Define the channel names and indices for Twente and Geneva
channel_names_geneva = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']
participant_ids = [f"s{i:02d}" for i in range(1, 6)]  # Assuming 5 participants
eeg_data_reordered = {}

# Load EEG data for each participant and reorder channels
for participant_id in participant_ids:
    file_path = f"eegs/{participant_id}.bdf"
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    
    # Keep only EEG channels
    raw_eeg = raw.pick_types(eeg=True)
    
    # Reorder EEG channels to Geneva order
    raw_reordered = raw_eeg.reorder_channels(channel_names_geneva)

    # Store reordered EEG data
    eeg_data_reordered[participant_id] = raw_reordered

# Print information about loaded and reordered data
for participant_id, raw_reordered in eeg_data_reordered.items():
    print(f"Participant ID: {participant_id}")
    print("Reordered EEG data shape:", raw_reordered._data.shape)


Check new channels and size

In [ ]:
import matplotlib.pyplot as plt

# Iterate through each participant
for participant_id, raw_reordered in eeg_data_reordered.items():
    print(f"Participant ID: {participant_id}")
    
    # Before reordering
    print("Channel names before reordering:", raw.info['ch_names'])
    
    # # Plot channel locations before reordering
    # fig_before = raw.plot_sensors(show_names=True, title="Channel Locations Before Reordering")
    # plt.show()
    
    # After reordering
    print("Channel names after reordering:", raw_reordered.info['ch_names'])
    
    # # Plot channel locations after reordering
    # fig_after = raw_reordered.plot_sensors(show_names=True, title="Channel Locations After Reordering")
    # plt.show()
    
    # Check data shape before and after reordering
    print("EEG data shape before reordering:", raw._data.shape)
    print("EEG data shape after reordering:", raw_reordered._data.shape)


Read and sort based on experiment_id the Participants_ratings csv

In [ ]:
df = pd.read_csv("participant_ratings.csv")
df_sorted = df.sort_values(by=['Participant_id', 'Experiment_id'])

df_sorted


Devide each eeg signal of each one of the participants into 40 60sec trials that represent the 40 experiments that took place

In [ ]:
# List to store trial data for each participant
participant_trials = {participant_id: [] for participant_id in participant_ids}

trial_info = df_sorted

# Iterate over each participant's data for the first 5 participants
for participant_id, participant_data in trial_info.groupby('Participant_id'):
    participant_id_str = f"s{participant_id:02d}"  # Convert participant_id to string format
    if participant_id_str not in participant_ids:
        continue

    # Get the raw EEG data for the current participant
    raw_data = eeg_data_reordered[participant_id_str]

    # Iterate over each trial for the current participant
    for index, trial in participant_data.iterrows():
        # Extract the start time of the trial (in seconds)
        start_time = trial['Start_time'] / 1e6  # Convert microseconds to seconds

        # Define the start and end time of the trial (assuming 1-minute duration)
        end_time = start_time + 60  # 1-minute duration

        # Extract the trial data based on the start and end time
        trial_data = raw_data.copy().crop(tmin=start_time, tmax=end_time)

        # Store the trial data in the list for the current participant
        participant_trials[participant_id_str].append(trial_data)

        # Print participant ID and trial information
        # print(participant_id_str, trial)


Print the number of trials

In [ ]:
# Get the number of participants
num_participants = len(participant_trials)
print(f"Number of participants: {num_participants}")

# Get the number of trials for each participant
for participant_id, trials in participant_trials.items():
    num_trials = len(trials)
    print(f"Participant {participant_id}: Number of trials = {num_trials}")

Print the trials of the first participant

In [ ]:
# Accessing the trial information DataFrame for the first participant
first_participant_trial_info = trial_info[trial_info['Participant_id'] == 1]

# Print the information of each trial
for index, trial_data in first_participant_trial_info.iterrows():
    print(f"Trial {index + 1} info:")
    print(trial_data)


We load the trials of the first participant into first_participant_trials and plot the first one

In [ ]:

first_participant_trials = participant_trials['s01']
first_trial_data = first_participant_trials[0]
first_trial_data.plot()

Plot the psd of the first trial

In [ ]:
first_trial_data.plot_psd()

PREPROCESSING

Bandpass filter 4->45 Hz

In [ ]:
low_freq = 4  # Lower cutoff frequency in Hz
high_freq = 45  # Upper cutoff frequency in Hz

# Iterate through each trial
for i, trial_data in enumerate(first_participant_trials):
    # Apply band-pass filter
    trial_data.filter(low_freq, high_freq)
    
    # Plot the EEG data for the first trial after filtering
    if i == 0:
        trial_data.plot()


Notch filter to cut off 50Hz and 60 Hz

In [ ]:
notch_freqs = [50, 60]  # Notch filter frequencies in Hz

# Iterate through each trial
for i, trial_data in enumerate(first_participant_trials):
    # Apply notch filters
    for freq in notch_freqs:
        trial_data.notch_filter(freqs=freq, verbose=True)
    
    # Plot the EEG data for the first trial after applying the notch filters
    if i == 0:
        trial_data.plot()


Set new sampling rate to 128 Hz

In [ ]:
# Define the new sampling rate
new_sampling_rate = 128

# Resample each trial for the first participant
for i, trial_data in enumerate(first_participant_trials):
    first_participant_trials[i] = trial_data.resample(new_sampling_rate, npad="auto")

# Plot the EEG data for the first trial after resampling
first_participant_trials[0].plot()


Print info

In [ ]:
print(first_participant_trials[0].ch_names)
first_participant_trials[0].info

Plot psd of the preprocessed first trial

In [ ]:
first_participant_trials[0].plot_psd()

In [ ]:
len(first_participant_trials[0])

Using ICA to remove artifacts

In [ ]:
from mne.preprocessing import ICA

# Set a fractional value for n_components
n_components_fraction = 32

# Initialize ICA with the extended Infomax method
ica = ICA(n_components=n_components_fraction, method='infomax')

# Fit the ICA model to your data
ica.fit(first_participant_trials[0])


In [ ]:
import matplotlib.pyplot as plt

# Plot each component separately
for i in range(len(ica.mixing_matrix_)):
    plt.figure()
    plt.plot(ica.mixing_matrix_[i])
    plt.title(f"Component {i+1}")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()


Applying common average reference

In [ ]:
for i, trial_data in enumerate(first_participant_trials):
    first_participant_trials[i] = trial_data.copy().set_eeg_reference('average', projection=True)
    first_participant_trials[i].apply_proj()

# Plot the EEG data for the first trial after applying CAR
first_participant_trials[0].plot()


Deviding each trial into 60 1sec subtrials

In [ ]:
import numpy as np

# Define the duration of each subtrial in seconds
subtrial_duration = 1  # 1 second

# Initialize a list to store the subtrials for all 40 trials of the first participant
first_participant_subtrials = []

# Iterate over each trial for the first participant
for trial_data in first_participant_trials:
    # Get the total duration of the trial in seconds
    trial_duration = trial_data.times[-1]

    # Calculate the number of subtrials
    num_subtrials = int(np.floor(trial_duration / subtrial_duration))

    # Initialize a list to store the subtrials for the current trial
    trial_subtrials = []

    # Iterate over each subtrial
    for i in range(num_subtrials):
        # Define the start and end time of the subtrial
        start_time = i * subtrial_duration
        end_time = (i + 1) * subtrial_duration

        # Extract the subtrial data
        subtrial_data = trial_data.copy().crop(tmin=start_time, tmax=end_time)

        # Append the subtrial data to the list
        trial_subtrials.append(subtrial_data)

    # Append the list of subtrials for the current trial to the list for all trials
    first_participant_subtrials.append(trial_subtrials)

# Now you have a nested list where first_participant_subtrials[i][j] represents the j-th subtrial of the i-th trial


In [ ]:
len(trial_subtrials)

Cut the first 3 seconds

In [ ]:
# Drop the first three subtrials from each trial's list of subtrials for all 40 trials of the first participant
for trial_subtrials in first_participant_subtrials:
    trial_subtrials = trial_subtrials[2:]


In [ ]:
len(trial_subtrials)

FEATURE EXTRACTION

Extract activity, complexity, mobility (Hjorth parameters)

In [ ]:
# Initialize lists to store computed Hjorth parameters for all trials of the first participant
all_activities = []
all_mobilities = []
all_complexities = []

# Iterate over each trial's list of subtrials
for trial_subtrials in first_participant_subtrials:
    # Initialize arrays to store computed Hjorth parameters for subtrials of the current trial
    trial_activities = []
    trial_mobilities = []
    trial_complexities = []

    # Compute Hjorth parameters for each subtrial of the current trial
    for subtrial_data in trial_subtrials:
        # Ensure the data is 2-dimensional
        subtrial_data_2d = subtrial_data.get_data().squeeze()

        # Compute the first derivative
        dy = np.diff(subtrial_data_2d, axis=1)

        # Compute the second derivative
        dyy = np.diff(subtrial_data_2d, n=2, axis=1)

        # Compute activity
        activity = np.var(subtrial_data_2d, axis=1)

        # Compute mobility
        mobility = np.sqrt(np.var(dy, axis=1) / activity)

        # Compute complexity
        complexity = np.sqrt(np.var(dyy, axis=1) / np.var(dy, axis=1)) / mobility

        # Append computed Hjorth parameters to respective arrays
        trial_activities.append(activity)
        trial_mobilities.append(mobility)
        trial_complexities.append(complexity)

    # Convert lists to arrays and append them to corresponding lists for all trials
    all_activities.append(np.array(trial_activities))
    all_mobilities.append(np.array(trial_mobilities))
    all_complexities.append(np.array(trial_complexities))

# Now all_activities, all_mobilities, and all_complexities contain the computed Hjorth parameters for each subtrial of all trials of the first participant


In [ ]:
# Print computed Hjorth parameters for each subtrial of the first trial
for i, (activity, mobility, complexity) in enumerate(zip(all_activities[0], all_mobilities[0], all_complexities[0]), start=1):
    print(f"Subtrial {i}:")
    print(f"Activity: {activity}")
    print(f"Mobility: {mobility}")
    print(f"Complexity: {complexity}")
    print()


In [ ]:
import numpy as np

# Initialize lists to store mean Hjorth parameters for each trial
trial_mean_activities = []
trial_mean_mobilities = []
trial_mean_complexities = []

# Iterate over each trial
for trial_activities, trial_mobilities, trial_complexities in zip(all_activities, all_mobilities, all_complexities):
    # Compute the mean across all subtrials for each Hjorth parameter individually
    mean_activity = np.mean(trial_activities, axis=0)
    mean_mobility = np.mean(trial_mobilities, axis=0)
    mean_complexity = np.mean(trial_complexities, axis=0)
    
    # Append the mean Hjorth parameters for the current trial to corresponding lists
    trial_mean_activities.append(mean_activity)
    trial_mean_mobilities.append(mean_mobility)
    trial_mean_complexities.append(mean_complexity)

# Convert lists to numpy arrays
trial_mean_activities = np.array(trial_mean_activities)
trial_mean_mobilities = np.array(trial_mean_mobilities)
trial_mean_complexities = np.array(trial_mean_complexities)

# Print the shape of the resulting arrays
print("Shape of trial_mean_activities:", trial_mean_activities.shape)
print("Shape of trial_mean_mobilities:", trial_mean_mobilities.shape)
print("Shape of trial_mean_complexities:", trial_mean_complexities.shape)


In [ ]:
# Compute the mean of the 32 values for each trial
trial_mean_activity_final = np.mean(trial_mean_activities, axis=1)
trial_mean_mobility_final = np.mean(trial_mean_mobilities, axis=1)
trial_mean_complexity_final = np.mean(trial_mean_complexities, axis=1)

# Print the mean of the 32 values for each trial
for i, (mean_activity, mean_mobility, mean_complexity) in enumerate(zip(trial_mean_activity_final, trial_mean_mobility_final, trial_mean_complexity_final), start=1):
    print(f"Trial {i}:")
    print(f"Mean Activity: {mean_activity}")
    print(f"Mean Mobility: {mean_mobility}")
    print(f"Mean Complexity: {mean_complexity}")
    print()


In [ ]:
import pandas as pd

outcome_df = pd.DataFrame()

# Initialize lists to store participant ID, experiment ID, and computed Hjorth parameters
participant_id = "1"  # Participant ID 's01' repeated 40 times
experiment_ids = list(range(1, 41))  # Experiment IDs from 1 to 40

# Create a dictionary to hold the features
data = {
    "Participant_id": participant_id,
    "Experiment_id": experiment_ids,
    "Activity": trial_mean_activity_final,
    "Mobility": trial_mean_mobility_final,
    "Complexity": trial_mean_complexity_final
}

# Create a DataFrame from the dictionary
outcome_df = pd.DataFrame(data)

# Print the DataFrame
print(outcome_df)


Extract statistical features

In [ ]:
# Initialize lists to store the features for all trials of the first participant
all_mean_values = []
all_std_values = []
all_max_values = []
all_min_values = []
all_rms_values = []
all_skewness_values = []
all_kurtosis_values = []
all_entropy_values = []

# Iterate over each trial's list of subtrials
for trial_subtrials in first_participant_subtrials:
    # Initialize lists to store the features for subtrials of the current trial
    mean_values = []
    std_values = []
    max_values = []
    min_values = []
    rms_values = []
    skewness_values = []
    kurtosis_values = []
    entropy_values = []

    # Iterate over each subtrial of the current trial
    for subtrial_data in trial_subtrials:
        # Flatten the subtrial data
        flat_data = subtrial_data.get_data().flatten()

        # Compute mean
        mean_values.append(np.mean(flat_data))

        # Compute standard deviation
        std_values.append(np.std(flat_data))

        # Compute maximum and minimum values
        max_values.append(np.max(flat_data))
        min_values.append(np.min(flat_data))

        # Compute root mean square (RMS)
        rms_values.append(np.sqrt(np.mean(flat_data**2)))

        # Compute skewness and kurtosis
        skewness_values.append(stats.skew(flat_data))
        kurtosis_values.append(stats.kurtosis(flat_data))

        # Compute sample entropy
        sample_entropy = ent.sample_entropy(flat_data, 2, 0.2*np.std(flat_data))
        entropy_values.append(sample_entropy)

    # Append computed features for subtrials of the current trial to corresponding lists
    all_mean_values.append(mean_values)
    all_std_values.append(std_values)
    all_max_values.append(max_values)
    all_min_values.append(min_values)
    all_rms_values.append(rms_values)
    all_skewness_values.append(skewness_values)
    all_kurtosis_values.append(kurtosis_values)
    all_entropy_values.append(entropy_values)




In [ ]:
# Print computed features for each subtrial of each trial
for i, (mean_trial, std_trial, max_trial, min_trial, rms_trial, skewness_trial, kurtosis_trial, entropy_trial) in enumerate(zip(all_mean_values, all_std_values, all_max_values, all_min_values, all_rms_values, all_skewness_values, all_kurtosis_values, all_entropy_values), start=1):
    print(f"Trial {i}:")
    for j, (mean_subtrial, std_subtrial, max_subtrial, min_subtrial, rms_subtrial, skewness_subtrial, kurtosis_subtrial, entropy_subtrial) in enumerate(zip(mean_trial, std_trial, max_trial, min_trial, rms_trial, skewness_trial, kurtosis_trial, entropy_trial), start=1):
        print(f"Subtrial {j}:")
        print(f"Mean: {mean_subtrial}")
        print(f"Standard Deviation: {std_subtrial}")
        print(f"Maximum: {max_subtrial}")
        print(f"Minimum: {min_subtrial}")
        print(f"Root Mean Square: {rms_subtrial}")
        print(f"Skewness: {skewness_subtrial}")
        print(f"Kurtosis: {kurtosis_subtrial}")
        print(f"Entropy: {entropy_subtrial}")
        print()


In [ ]:
# Initialize lists to store the mean values of features for each trial
trial_mean_values = []
trial_std_values = []
trial_max_values = []
trial_min_values = []
trial_rms_values = []
trial_skewness_values = []
trial_kurtosis_values = []
trial_entropy_values = []

# Iterate over each trial's features
for trial_features in zip(all_mean_values, all_std_values, all_max_values, all_min_values, all_rms_values, all_skewness_values, all_kurtosis_values, all_entropy_values):
    # Compute the mean value of each feature across all subtrials for the current trial
    trial_mean_values.append(np.mean(trial_features[0]))
    trial_std_values.append(np.mean(trial_features[1]))
    trial_max_values.append(np.mean(trial_features[2]))
    trial_min_values.append(np.mean(trial_features[3]))
    trial_rms_values.append(np.mean(trial_features[4]))
    trial_skewness_values.append(np.mean(trial_features[5]))
    trial_kurtosis_values.append(np.mean(trial_features[6]))
    trial_entropy_values.append(np.mean(trial_features[7]))

# Print the mean values of features for each trial
for i, (mean_val, std_val, max_val, min_val, rms_val, skewness_val, kurtosis_val, entropy_val) in enumerate(zip(trial_mean_values, trial_std_values, trial_max_values, trial_min_values, trial_rms_values, trial_skewness_values, trial_kurtosis_values, trial_entropy_values), start=1):
    print(f"Trial {i}:")
    print(f"Mean: {mean_val}")
    print(f"Standard Deviation: {std_val}")
    print(f"Maximum: {max_val}")
    print(f"Minimum: {min_val}")
    print(f"Root Mean Square: {rms_val}")
    print(f"Skewness: {skewness_val}")
    print(f"Kurtosis: {kurtosis_val}")
    print(f"Entropy: {entropy_val}")
    print()


In [ ]:


# # Create a list to hold the participant ID (s01) and experiment ID (1 to 40) for each trial
# participant_ids = ["s01"] * 40
# experiment_ids = list(range(1, 41))

new_data = {
    "Mean": trial_mean_values,
    "Standard_Deviation": trial_std_values,
    "Maximum": trial_max_values,
    "Minimum": trial_min_values,
    "Root_Mean_Square": trial_rms_values,
    "Skewness": trial_skewness_values,
    "Kurtosis": trial_kurtosis_values,
    "Entropy": trial_entropy_values
}


# Update the existing dictionary with new features
data.update(new_data)

# Create a DataFrame from the updated dictionary
outcome_df = pd.DataFrame(data)

# Print the DataFrame
print(outcome_df)


In [ ]:
outcome_df

Add valence and arousal columns

In [ ]:
import pandas as pd

# Load the CSV file
csv_data = pd.read_csv("participant_ratings.csv")

# Convert Participant_id column in CSV data to string type
csv_data['Participant_id'] = csv_data['Participant_id'].astype(str)

# Merge the CSV data with our DataFrame based on participant_id and experiment_id
merged_data = pd.merge(outcome_df, csv_data[['Participant_id', 'Experiment_id', 'Valence', 'Arousal']], 
                       how='left', on=['Participant_id', 'Experiment_id'])

# Print the merged DataFrame
print(merged_data)



In [ ]:
merged_data

Compute energy and differential entropy for each band

In [ ]:
import numpy as np
from scipy.fft import fft
from scipy.stats import entropy


fs = 128  

def compute_energy(signal, sampling_rate, frequency_band=None):
    
    fft_result = fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)
    if frequency_band is not None:
        band_indices = np.where((freqs >= frequency_band[0]) & (freqs <= frequency_band[1]))[0]
        band_fft = fft_result[band_indices]
    else:
        band_fft = fft_result
    energy = np.sum(np.abs(band_fft)**2) / len(signal)
    return energy


def compute_differential_entropy(signal, sampling_rate, frequency_band):
    
    fft_result = fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)
    band_indices = np.where((freqs >= frequency_band[0]) & (freqs <= frequency_band[1]))[0]
    band_fft = fft_result[band_indices]
    band_psd = np.abs(band_fft)**2 / len(signal)
    band_pdf = band_psd / np.sum(band_psd)
    diff_entropy = entropy(band_pdf)
    return diff_entropy

# Define frequency bands
theta_band = (4, 8)
alpha_band = (8, 14)
beta_band = (14, 31)
gamma_band = (31, 45)

# Initialize lists to store energy values for each frequency band
theta_energy_values = []
alpha_energy_values = []
beta_energy_values = []
gamma_energy_values = []

# Initialize lists to store differential entropy values for each frequency band
theta_diff_entropy_values = []
alpha_diff_entropy_values = []
beta_diff_entropy_values = []
gamma_diff_entropy_values = []
t=0
# Loop through each trial
for trial_subtrials in first_participant_subtrials:
    print(t)
    t += 1
    # Initialize lists to store energy and differential entropy for each channel
    trial_theta_energy_values = []
    trial_alpha_energy_values = []
    trial_beta_energy_values = []
    trial_gamma_energy_values = []

    trial_theta_diff_entropy_values = []
    trial_alpha_diff_entropy_values = []
    trial_beta_diff_entropy_values = []
    trial_gamma_diff_entropy_values = []

    # Loop through each subtrial in the trial
    for subtrial_data in trial_subtrials:
        # Initialize lists to store energy and differential entropy for each channel
        channel_theta_energy_values = []
        channel_alpha_energy_values = []
        channel_beta_energy_values = []
        channel_gamma_energy_values = []

        channel_theta_diff_entropy_values = []
        channel_alpha_diff_entropy_values = []
        channel_beta_diff_entropy_values = []
        channel_gamma_diff_entropy_values = []

        # Loop through each channel in the subtrial
        for channel_data_tuple in subtrial_data:
            # Initialize list to store flattened channel data
            flattened_channel_data = []

            # Loop through each array within the channel data tuple
            for data_array in channel_data_tuple:
                # If the array has more than one dimension, flatten it
                if data_array.ndim > 1:
                    flattened_channel_data.append(data_array.flatten())
                elif data_array.ndim == 1:
                    flattened_channel_data.append(data_array)  # Add 1-dimensional arrays as is
                else:
                    # Skip zero-dimensional arrays
                    continue

            # Concatenate flattened channel data into a single array if there's any data
            if flattened_channel_data:
                channel_data = np.concatenate(flattened_channel_data)
            else:
                # If no data was found, skip computation for this channel
                continue

            # Compute energy in different frequency bands for the channel
            theta_energy = compute_energy(channel_data, fs)
            alpha_energy = compute_energy(channel_data, fs)
            beta_energy = compute_energy(channel_data, fs)
            gamma_energy = compute_energy(channel_data, fs)

            # Compute differential entropy in different frequency bands for the channel
            theta_diff_entropy = compute_differential_entropy(channel_data, fs, theta_band)
            alpha_diff_entropy = compute_differential_entropy(channel_data, fs, alpha_band)
            beta_diff_entropy = compute_differential_entropy(channel_data, fs, beta_band)
            gamma_diff_entropy = compute_differential_entropy(channel_data, fs, gamma_band)

            # Append results to channel lists
            channel_theta_energy_values.append(theta_energy)
            channel_alpha_energy_values.append(alpha_energy)
            channel_beta_energy_values.append(beta_energy)
            channel_gamma_energy_values.append(gamma_energy)

            channel_theta_diff_entropy_values.append(theta_diff_entropy)
            channel_alpha_diff_entropy_values.append(alpha_diff_entropy)
            channel_beta_diff_entropy_values.append(beta_diff_entropy)
            channel_gamma_diff_entropy_values.append(gamma_diff_entropy)

        # Append channel-wise results to trial lists
        trial_theta_energy_values.append(channel_theta_energy_values)
        trial_alpha_energy_values.append(channel_alpha_energy_values)
        trial_beta_energy_values.append(channel_beta_energy_values)
        trial_gamma_energy_values.append(channel_gamma_energy_values)

        trial_theta_diff_entropy_values.append(channel_theta_diff_entropy_values)
        trial_alpha_diff_entropy_values.append(channel_alpha_diff_entropy_values)
        trial_beta_diff_entropy_values.append(channel_beta_diff_entropy_values)
        trial_gamma_diff_entropy_values.append(channel_gamma_diff_entropy_values)

    # Append trial-wise results to overall lists
    theta_energy_values.append(trial_theta_energy_values)
    alpha_energy_values.append(trial_alpha_energy_values)
    beta_energy_values.append(trial_beta_energy_values)
    gamma_energy_values.append(trial_gamma_energy_values)

    theta_diff_entropy_values.append(trial_theta_diff_entropy_values)
    alpha_diff_entropy_values.append(trial_alpha_diff_entropy_values)
    beta_diff_entropy_values.append(trial_beta_diff_entropy_values)
    gamma_diff_entropy_values.append(trial_gamma_diff_entropy_values)


# Compute mean energy and differential entropy across trials and subtrials
theta_energy_mean = np.mean(theta_energy_values, axis=(1, 2 ))
alpha_energy_mean = np.mean(alpha_energy_values, axis=(1, 2))
beta_energy_mean = np.mean(beta_energy_values, axis=(1, 2))
gamma_energy_mean = np.mean(gamma_energy_values, axis=(1, 2))

theta_diff_entropy_mean = np.mean(theta_diff_entropy_values, axis=(1, 2))
alpha_diff_entropy_mean = np.mean(alpha_diff_entropy_values, axis=(1, 2))
beta_diff_entropy_mean = np.mean(beta_diff_entropy_values, axis=(1, 2))
gamma_diff_entropy_mean = np.mean(gamma_diff_entropy_values, axis=(1, 2))

# Print shapes after computing mean
print("\nShapes after computing mean:")
print("theta_energy_mean shape:", theta_energy_mean.shape)
print("alpha_energy_mean shape:", alpha_energy_mean.shape)
print("beta_energy_mean shape:", beta_energy_mean.shape)
print("gamma_energy_mean shape:", gamma_energy_mean.shape)
print("theta_diff_entropy_mean shape:", theta_diff_entropy_mean.shape)
print("alpha_diff_entropy_mean shape:", alpha_diff_entropy_mean.shape)
print("beta_diff_entropy_mean shape:", beta_diff_entropy_mean.shape)
print("gamma_diff_entropy_mean shape:", gamma_diff_entropy_mean.shape)


In [ ]:
# Print the mean values for each trial
for i in range(len(theta_energy_mean)):
    print(f"Trial {i+1}:")
    print(f"\tTheta Energy: {theta_energy_mean[i]}")
    print(f"\tAlpha Energy: {alpha_energy_mean[i]}")
    print(f"\tBeta Energy: {beta_energy_mean[i]}")
    print(f"\tGamma Energy: {gamma_energy_mean[i]}")
    print(f"\tTheta Diff. Entropy: {theta_diff_entropy_mean[i]}")
    print(f"\tAlpha Diff. Entropy: {alpha_diff_entropy_mean[i]}")
    print(f"\tBeta Diff. Entropy: {beta_diff_entropy_mean[i]}")
    print(f"\tGamma Diff. Entropy: {gamma_diff_entropy_mean[i]}")


In [ ]:
# Create a dictionary to hold the mean values
new_data = {
    "Mean Theta Energy": theta_energy_mean,
    "Mean Alpha Energy": alpha_energy_mean,
    "Mean Beta Energy": beta_energy_mean,
    "Mean Gamma Energy": gamma_energy_mean,
    "Mean Theta Differential Entropy": theta_diff_entropy_mean,
    "Mean Alpha Differential Entropy": alpha_diff_entropy_mean,
    "Mean Beta Differential Entropy": beta_diff_entropy_mean,
    "Mean Gamma Differential Entropy": gamma_diff_entropy_mean
}

# Update the existing dictionary with new features
data.update(new_data)

# Create a DataFrame from the updated dictionary
outcome_df = pd.DataFrame(data)

# Print the DataFrame
outcome_df.columns


Compute Wavelet features

In [ ]:
fs = 128  # Sampling rate of the EEG signal

def compute_features(signal):
    
    # Perform wavelet decomposition
    coeffs = pywt.wavedec(signal, wavelet='db4', level=5)

    # Initialize list to store features
    features = []

    # Extract statistical features from wavelet coefficients
    for coeff in coeffs:
        features.extend([np.mean(coeff), np.var(coeff), np.std(coeff), np.max(coeff), np.min(coeff)])

    return features

# Initialize lists to store features for each trial
wavelet_features_per_trial = []
t=0
# Loop through each trial
for trial_subtrials in first_participant_subtrials:
    print(t)
    t+=1
    # Initialize lists to store features for each subtrial within the trial
    wavelet_features_per_subtrial = []

    # Loop through each subtrial in the trial
    for subtrial_data in trial_subtrials:
        # Initialize lists to store features for each channel within the subtrial
        wavelet_features_per_channel = []

        # Loop through each channel in the subtrial
        for channel_data_tuple in subtrial_data:
            # Initialize an empty list to store flattened channel data
            flattened_channel_data = []

            # Loop through each array within the channel data tuple
            for data_array in channel_data_tuple:
                # If the array has more than one dimension, flatten it
                if data_array.ndim > 1:
                    flattened_channel_data.append(data_array.flatten())
                else:
                    flattened_channel_data.append(data_array)

            # Concatenate the flattened channel data into a single one-dimensional array
            channel_data = np.concatenate(flattened_channel_data)

            # Compute features for the channel data
            features = compute_features(channel_data)

            # Append features to the list
            wavelet_features_per_channel.append(features)

        # Append features for all channels in the subtrial to the list
        wavelet_features_per_subtrial.append(wavelet_features_per_channel)

    # Append features for all subtrials in the trial to the list
    wavelet_features_per_trial.append(wavelet_features_per_subtrial)

# Convert list of features to NumPy array
wavelet_features_per_trial = np.array(wavelet_features_per_trial)

# Now 'wavelet_features_per_trial' contains the extracted features from the wavelet coefficients for each trial
print(wavelet_features_per_trial)


In [ ]:
wavelet_features_per_trial_mean =np.mean(wavelet_features_per_trial, axis=(1, 2 ))

In [ ]:
# Print the shape of wavelet_features_per_trial
print("Shape of wavelet_features_per_trial:", wavelet_features_per_trial_mean.shape)


In [ ]:
# Print the mean values for each trial
for i in range(len(wavelet_features_per_trial_mean)):
    print(f"Trial {i+1}:")
    print(f"\tWavelet Features: {wavelet_features_per_trial_mean[i]}")


In [ ]:
new_wavelet_data = {}

# Assuming you have 30 features
for i in range(30):
    new_wavelet_data[f"Mean Wavelet Feature {i+1}"] = wavelet_features_per_trial_mean[:, i]

# Update the existing dictionary with new features
data.update(new_wavelet_data)

# Create a DataFrame from the updated dictionary
outcome_df = pd.DataFrame(data)

# Print the DataFrame
outcome_df


Compute Wavelet entropy

In [ ]:
fs = 128  # Sampling rate of the EEG signal

def compute_wavelet_entropy(signal):
   
    # Perform wavelet decomposition with level 4 and zero-padding
    coeffs = pywt.wavedec(signal, wavelet='db4', level=4, mode='zero')

    # Initialize list to store entropy values
    entropy_values = []

    # Calculate entropy for each wavelet coefficient
    for coeff in coeffs:
        # Compute probability distribution
        prob_distribution = np.abs(coeff) / np.sum(np.abs(coeff))
        
        # Avoid division by zero by adding a small epsilon value
        prob_distribution[prob_distribution == 0] = 1e-10

        # Compute entropy
        entropy = -np.sum(prob_distribution * np.log2(prob_distribution))

        # Append entropy to the list
        entropy_values.append(entropy)

    return np.array(entropy_values)


# Initialize lists to store entropy values for each trial
wavelet_entropy_values_per_trial = []
t = 0
# Loop through each trial
for trial_subtrials in first_participant_subtrials:
    print(t)
    t += 1
    # Initialize lists to store entropy values for each subtrial within the trial
    wavelet_entropy_values_per_subtrial = []

    # Loop through each subtrial in the trial
    for subtrial_data in trial_subtrials:
        # Initialize lists to store entropy values for each channel within the subtrial
        wavelet_entropy_values_per_channel = []

        # Loop through each channel in the subtrial
        for channel_data_tuple in subtrial_data:
            # Initialize an empty list to store flattened channel data
            flattened_channel_data = []

            # Loop through each array within the channel data tuple
            for data_array in channel_data_tuple:
                # If the array has more than one dimension, flatten it
                if data_array.ndim > 1:
                    flattened_channel_data.append(data_array.flatten())
                else:
                    flattened_channel_data.append(data_array)

            # Concatenate the flattened channel data into a single one-dimensional array
            channel_data = np.concatenate(flattened_channel_data)

            # Compute wavelet entropy for the channel data
            entropy_values = compute_wavelet_entropy(channel_data)

            # Append entropy values to the list
            wavelet_entropy_values_per_channel.append(entropy_values)

        # Append entropy values for all channels in the subtrial to the list
        wavelet_entropy_values_per_subtrial.append(wavelet_entropy_values_per_channel)

    # Append entropy values for all subtrials in the trial to the list
    wavelet_entropy_values_per_trial.append(wavelet_entropy_values_per_subtrial)

# Convert list of entropy values to NumPy array
wavelet_entropy_values_per_trial = np.array(wavelet_entropy_values_per_trial)

# Now 'wavelet_entropy_values_per_trial' contains the computed wavelet entropy values for each trial
print(wavelet_entropy_values_per_trial)


In [ ]:
# Print the shape of wavelet_features_per_trial
print("Shape of wavelet_features_per_trial:", wavelet_entropy_values_per_trial.shape)

In [ ]:
wavelet_entropy_values_per_trial_mean =np.mean(wavelet_entropy_values_per_trial, axis=(1, 2 ))

In [ ]:
# Print the shape of wavelet_features_per_trial
print("Shape of wavelet_features_per_trial:", wavelet_entropy_values_per_trial_mean.shape)

In [ ]:
# Print the mean values for each trial
for i in range(len(wavelet_entropy_values_per_trial_mean)):
    print(f"Trial {i+1}:")
    print(f"\tWavelet Features: {wavelet_entropy_values_per_trial_mean[i]}")


In [ ]:
new_wavelet_data = {}

# Assuming you have 30 features
for i in range(5):
    new_wavelet_data[f"Mean Entropy Value {i+1}"] = wavelet_entropy_values_per_trial_mean[:, i]

# Update the existing dictionary with new features
data.update(new_wavelet_data)

# Create a DataFrame from the updated dictionary
outcome_df = pd.DataFrame(data)

# Print the DataFrame
outcome_df

In [ ]:
def compute_auto_correlation(signal):
    # Compute auto-correlation
    auto_corr = np.correlate(signal, signal, mode='full')
    # Extract relevant features from auto-correlation
    features = [
        np.mean(auto_corr),
        np.max(auto_corr),
        np.min(auto_corr),
        np.std(auto_corr)
    ]
    return features

def compute_zero_crossing_rate(signal):
    # Count zero crossings
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    # Compute zero crossing rate
    zero_crossing_rate = len(zero_crossings) / len(signal)
    return [zero_crossing_rate]

# Initialize lists to store mean features for each trial
mean_auto_corr_features_per_trial = []
mean_zero_crossing_rate_features_per_trial = []
t=0

# Loop through each trial
for trial_subtrials in first_participant_subtrials:
    auto_corr_features = []
    zero_crossing_rate_features = []
    t+=1
    print(t)

    # Loop through each subtrial in the trial
    for subtrial_data in trial_subtrials:
        # Loop through each channel in the subtrial
        for channel_data_tuple in subtrial_data:
            # Initialize list to store flattened channel data
            flattened_channel_data = []

            # Loop through each array within the channel data tuple
            for data_array in channel_data_tuple:
                # If the array has more than one dimension, flatten it
                if data_array.ndim > 1:
                    flattened_channel_data.append(data_array.flatten())
                elif data_array.ndim == 1:
                    flattened_channel_data.append(data_array)  # Add 1-dimensional arrays as is
                else:
                    # Skip zero-dimensional arrays
                    continue

            # Concatenate flattened channel data into a single array if there's any data
            if flattened_channel_data:
                channel_data = np.concatenate(flattened_channel_data)
            else:
                # If no data was found, skip computation for this channel
                continue

            # Compute auto-correlation features for the channel data
            auto_corr_features.append(compute_auto_correlation(channel_data))

            # Compute zero crossing rate features for the channel data
            zero_crossing_rate_features.append(compute_zero_crossing_rate(channel_data))

    # Convert lists of features to NumPy arrays for easier mean calculation
    auto_corr_features = np.array(auto_corr_features)
    zero_crossing_rate_features = np.array(zero_crossing_rate_features)

    # Compute mean features for the current trial
    mean_auto_corr_features = np.mean(auto_corr_features, axis=0)
    mean_zero_crossing_rate_features = np.mean(zero_crossing_rate_features, axis=0)

    # Store the mean features
    mean_auto_corr_features_per_trial.append(mean_auto_corr_features)
    mean_zero_crossing_rate_features_per_trial.append(mean_zero_crossing_rate_features)

# Convert lists of mean features to NumPy arrays
mean_auto_corr_features_per_trial = np.array(mean_auto_corr_features_per_trial)
mean_zero_crossing_rate_features_per_trial = np.array(mean_zero_crossing_rate_features_per_trial)

# Print mean features for each trial
print("Mean Auto-correlation Features per Trial:", mean_auto_corr_features_per_trial)
print("Mean Zero Crossing Rate Features per Trial:", mean_zero_crossing_rate_features_per_trial)


In [ ]:
new_data = {}

# Assuming you have 30 features
for i in range(4):
    new_data[f"Mean Auto-correlation feature {i+1}"] = mean_auto_corr_features_per_trial[:, i]

# Update the existing dictionary with new features
data.update(new_data)

# Create a DataFrame from the updated dictionary
merged_data = pd.DataFrame(data)

# Print the DataFrame
merged_data

In [ ]:
# Convert the 2D array to a 1D array
flattened_zero_crossing_rate_features = mean_zero_crossing_rate_features_per_trial.flatten()

# Add the flattened array to the dictionary
new_data = {
    "Zero Crossing Rate": flattened_zero_crossing_rate_features
}

# Update the existing dictionary with new features
data.update(new_data)

# Create a DataFrame from the updated dictionary
outcome_df = pd.DataFrame(data)

outcome_df


CLASSIFICATION


In [158]:


# Define a function to classify valence and arousal
def classify_value(value):
    return 1 if value > 5 else 0

# Apply the function to create the new columns
merged_data['Valence_class'] = merged_data['Valence'].apply(classify_value)
merged_data['Arousal_class'] = merged_data['Arousal'].apply(classify_value)

# Display the first few rows to check the new columns
print(merged_data.head())


  Participant_id  Experiment_id      Activity  Mobility  Complexity  \
0              1              1  2.551833e-11  0.656182    1.562014   
1              1              2  2.850130e-11  0.641845    1.566043   
2              1              3  2.687303e-11  0.647403    1.563928   
3              1              4  2.338460e-11  0.714100    1.570787   
4              1              5  2.334399e-11  0.723316    1.570053   

           Mean  Standard_Deviation   Maximum   Minimum  Root_Mean_Square  \
0  5.341950e-24            0.000005  0.000019 -0.000020          0.000005   
1 -9.487526e-24            0.000005  0.000021 -0.000022          0.000005   
2 -4.103842e-25            0.000005  0.000021 -0.000021          0.000005   
3  2.643152e-24            0.000005  0.000023 -0.000023          0.000005   
4 -1.905852e-24            0.000005  0.000023 -0.000023          0.000005   

   ...  Mean Entropy Value 1  Mean Entropy Value 2  Mean Entropy Value 3  \
0  ...              2.842203      

In [161]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import classification_report, accuracy_score

# Assuming `merged_data` is already defined and contains the dataset
# Separate features and target
X = merged_data.drop(['Valence', 'Arousal', 'Valence_class', 'Arousal_class'], axis=1)
y = merged_data['Valence_class']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into labeled and unlabeled data
# For this example, let's assume we use 30% of the data as labeled
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X, y, test_size=0.7, random_state=42, stratify=y)

# Replace a portion of the labels with -1 to indicate unlabeled data
y_unlabeled[:] = -1

# Combine the labeled and unlabeled datasets
X_combined = np.vstack((X_labeled, X_unlabeled))
y_combined = np.concatenate((y_labeled, y_unlabeled))

print(f"Number of labeled samples: {len(y_labeled)}")
print(f"Number of unlabeled samples: {len(y_unlabeled)}")

# Initialize and fit the Label Spreading model
label_spread = LabelSpreading(kernel='rbf', alpha=0.2)
label_spread.fit(X_combined, y_combined)

# Predict the labels for all data
y_transduced = label_spread.transduction_

# Extract predictions for the labeled data
y_pred_labeled = y_transduced[:len(y_labeled)]

# Evaluate the model
print("Classification report for Label Spreading:")
print(classification_report(y_labeled, y_pred_labeled))
print("Accuracy:", accuracy_score(y_labeled, y_pred_labeled))

# Create a DataFrame to visualize the results
columns = merged_data.drop(['Valence', 'Arousal', 'Valence_class', 'Arousal_class'], axis=1).columns
df = pd.DataFrame(X_combined, columns=columns)
df['True Label'] = y_combined
df['Predicted Label'] = y_transduced

print(df)


Number of labeled samples: 12
Number of unlabeled samples: 28
Classification report for Label Spreading:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

Accuracy: 1.0
    Participant_id  Experiment_id  Activity  Mobility  Complexity      Mean  \
0              0.0      -0.649722 -1.321360  0.932445    0.206650 -1.318546   
1              0.0       1.169500 -0.913933  0.278778   -1.326372  0.466494   
2              0.0      -0.043315 -0.397352  1.435491    1.703248  0.701949   
3              0.0       0.303204 -0.335891  1.586976    1.649586  1.175066   
4              0.0       0.216574  1.500910 -1.170471    0.221820  0.725495   
5              0.0      -1.429389 -0.258081  1.765899    0.725503  0.881483   
6

In [162]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Perform cross-validation
scores = cross_val_score(label_spread, X_combined, y_combined, cv=cv, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())


Cross-validation scores: [0.25  0.125 0.25  0.125 0.   ]
Mean accuracy: 0.15


In [163]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Initialize the SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(svm_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())


Cross-validation scores: [0.625 0.75  0.75  0.875 0.5  ]
Mean accuracy: 0.7


In [164]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation
cv_scores_rf = cross_val_score(rf_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores (Random Forest):", cv_scores_rf)
print("Mean accuracy (Random Forest):", cv_scores_rf.mean())


Cross-validation scores (Random Forest): [0.625 0.75  0.75  0.625 0.5  ]
Mean accuracy (Random Forest): 0.65


In [165]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Perform cross-validation
cv_scores_knn = cross_val_score(knn_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores (KNN):", cv_scores_knn)
print("Mean accuracy (KNN):", cv_scores_knn.mean())


Cross-validation scores (KNN): [0.625 0.75  0.625 0.875 0.5  ]
Mean accuracy (KNN): 0.675


c:\Users\Odysseas\git\Emotion_Recognition\ml\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Odysseas\git\Emotion_Recognition\ml\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [166]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier()

# Perform cross-validation
cv_scores_xgb = cross_val_score(xgb_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores (XGBoost):", cv_scores_xgb)
print("Mean accuracy (XGBoost):", cv_scores_xgb.mean())


Cross-validation scores (XGBoost): [0.625 0.5   0.5   0.625 0.625]
Mean accuracy (XGBoost): 0.575


In [167]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

# Initialize the GBM classifier
gbm_classifier = GradientBoostingClassifier()

# Perform cross-validation
cv_scores_gbm = cross_val_score(gbm_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores (GBM):", cv_scores_gbm)
print("Mean accuracy (GBM):", cv_scores_gbm.mean())


Cross-validation scores (GBM): [0.25  0.625 0.625 0.75  0.75 ]
Mean accuracy (GBM): 0.6


In [168]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the Logistic Regression classifier
log_reg_classifier = LogisticRegression()

# Perform cross-validation
cv_scores_log_reg = cross_val_score(log_reg_classifier, X, y, cv=5)

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores (Logistic Regression):", cv_scores_log_reg)
print("Mean accuracy (Logistic Regression):", cv_scores_log_reg.mean())


Cross-validation scores (Logistic Regression): [0.75  0.375 1.    0.75  0.625]
Mean accuracy (Logistic Regression): 0.7


In [172]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Reshape X to mimic image data
X_2d = X.reshape(X.shape[0], X.shape[1], 1, 1)  # Assuming X.shape[1] is the number of features

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_2d, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))


model = Sequential([
    Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]),
    Conv2D(64, (1, 1), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10


c:\Users\Odysseas\git\Emotion_Recognition\ml\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5312 - loss: 0.6912 - val_accuracy: 0.6250 - val_loss: 0.6735
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.7188 - loss: 0.6192 - val_accuracy: 0.6250 - val_loss: 0.6845
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.7188 - loss: 0.5918 - val_accuracy: 0.6250 - val_loss: 0.7037
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.7188 - loss: 0.5815 - val_accuracy: 0.6250 - val_loss: 0.7118
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7500 - loss: 0.5733 - val_accuracy: 0.6250 - val_loss: 0.7124
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.7500 - loss: 0.5630 - val_accuracy: 0.6250 - val_loss: 0.7120
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7500 - loss: 0.5498 - val_accuracy: 0.6250 - val_loss: 0.7144
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7500 - loss: 0.5358 - val_accuracy: 0.6250 - val_loss: 0.7210
Epoch 9/1